In [3]:
%pip install protobuf==3.20.1
%pip install -q transformers sentencepiece

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [55]:
import os
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

<module 'preprocessing_utils' from '/usr/net/zinovyee.hub/IRTG/Encode-the-Qode/Quantlet/Create_description/preprocessing_utils.py'>

In [56]:
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_19032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

CLEAN_UP = True

In [57]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_large_source_code_summarization_python_multitask",
    skip_special_tokens=False),
    device=0
)

/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:921: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [61]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

nl_description_list = []
for batch_idx, samples in tqdm(enumerate(data_loader)):
      try:
            nl_description = pipeline([codeline[0] for codeline in samples])
            if CLEAN_UP:
                  nl_description = [re.sub(r'[^a-zA-Z0-9\s]', '', summary['summary_text']) for summary in nl_description]
                  nl_description = [re.sub(r'\s+', ' ', summary).strip() for summary in nl_description]
                  nl_description = [summary for summary in nl_description if len(summary) > 0]
      except:
            nl_description = []
      nl_description_list.append({batch_idx : nl_description})

62it [01:15,  1.38it/s]Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
262it [05:07,  1.42it/s]Your max_length is set to 20, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
265it [05:09,  1.64it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
332it [07:08,  1.37s/it]Your max_length is set to 20, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
479it [09:25,  1.90it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
492it [09:35,  1.19it/s]Your max_length is set to 20, but you input_length is only 18. You might consider decreasing max_

In [63]:
len(nl_description_list)

8379

In [64]:
descr_Qs_file = '../../data/preprocessed/Quantlet/Descriptions_Qs_19032023.pkl'

In [65]:
with open(descr_Qs_file, 'wb') as f:
        pickle.dump(nl_description_list, f)